In [16]:
%load_ext autoreload
%autoreload 2


import pandas as pd
from exploration.import_graph import AcademicGraph
from exploration.graph_features import process_features

import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
G = AcademicGraph()

domains, graphs, graph_features = G.segemented_graphs()

Preparing field: Applied Sciences
Preparing field: Education
Preparing field: Engineering
Preparing field: Humanities
Preparing field: Mathematics and Computing
Preparing field: Medicine and Health
Preparing field: Natural Sciences
Preparing field: Social Sciences


In [7]:

feats={}
for domain, g in zip(domains, graphs):
    feats[domain] = process_features(g)

In [18]:
np.mean(list(feats["Applied Sciences"]["clustering_coeff"].values()))



0.00317792955146534

In [20]:
feats["Mathematics and Computing"]

{'num_edges': nan,
 'num_nodes': 9418,
 'average_degree_connectivity': {320: 232.23333333333332,
  352: 224.26420454545453,
  430: 202.92558139534884,
  20: 360.4576923076923,
  131: 357.03053435114504,
  34: 343.8627450980392,
  22: 350.81060606060606,
  513: 176.36257309941521,
  477: 185.64779874213838,
  489: 174.22903885480574,
  218: 283.7079510703364,
  360: 216.1611111111111,
  417: 204.55995203836932,
  259: 252.9266409266409,
  330: 237.65454545454546,
  506: 173.0395256916996,
  546: 160.52380952380952,
  476: 183.59453781512605,
  474: 183.0084388185654,
  405: 209.86172839506173,
  456: 191.5482456140351,
  355: 224.66478873239436,
  531: 174.06214689265536,
  454: 197.4339207048458,
  529: 169.34971644612477,
  482: 181.60719225449515,
  197: 325.88832487309645,
  511: 172.51859099804307,
  186: 291.9139784946237,
  429: 198.17016317016316,
  487: 179.86447638603696,
  255: 246.86928104575162,
  323: 244.2879256965944,
  351: 235.55270655270655,
  144: 292.6828703703704,
